## 過去のNotebook

[日本語][EDA] JP Tokyo Stock データ概要を確認 その1 https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-1 <br>
[日本語][EDA] JP Tokyo Stock データ概要を確認 その2[移動平均線等] https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-2 <br>
[日本語][EDA] JP Tokyo Stock データ概要を確認 その3[ボリンジャーバンド等] https://www.kaggle.com/code/tatsuyafujii/eda-jp-tokyo-stock-3/notebook <br>


In [ ]:
!pip install japanize_matplotlib

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
import optuna.integration.lightgbm as lgb
import seaborn as sns
import japanize_matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def MA(series, window=25):
    return series.rolling(window).mean()

def DMA(series, window=25):
    return series/MA(series, window) - 1

def divergence(series, window=25):
    std = series.rolling(window).std()
    mean = series.rolling(window).mean()
    return (series-mean) / std    

def rsi(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:s[s>0].sum()/abs(s).sum())

def stochastic(series, k=14, n=3, m=3):
    _min = series.rolling(k).min()
    _max = series.rolling(k).max()
    _k = (series - _min)/(_max - _min)
    _d1 = _k.rolling(n).mean()
    _d2 = _d1.rolling(m).mean()
    return _k, _d1, _d2

## メインデータ

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv", parse_dates=["Date"])
print(prices.shape)
prices.head()

## 概観

In [ ]:
## 翌日、翌々日の終値を取得（Target候補）
df = prices.copy()
df["Close_adj"] = df.groupby("SecuritiesCode").apply(lambda d:d["Close"]/d["AdjustmentFactor"].cumprod().shift().fillna(1)).reset_index("SecuritiesCode",drop=True)
df["翌日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-1)
df["翌々日終値"] = df.groupby("SecuritiesCode")["Close_adj"].shift(-2)
df["R"] = (df["翌々日終値"] - df["翌日終値"]) / df["翌日終値"]
(df["R"]-df["Target"]).describe()

In [ ]:
## ある一つの銘柄に注目して確認
d = df[df["SecuritiesCode"]==4923].copy()

sns.lineplot(x=d.Date, y=d.R, label="収益率")
sns.lineplot(x=d.Date, y=d.Target, label="Target")
plt.show()

## サイコロジカルライン
値上がり（同値も含むとする）の確率。<br>
RSIと相関が高い指標であるが、幅を情報として持たない点が異なる。一概にどちらが良いとはいえない。<br>
大きくなりすぎると戻しの確率が大きくなる。一般に75％、25％程度を使うが、銘柄によって大きく異なる。<br>

### 考察
・下は30程度を下回る事が多く、下回ると戻しになりやすそうだが、上は70はほぼ超えていない。<br>
・トレンド方向と直感的に逆な箇所もあり、値上がりと値下がりの非対称性によるものと思われる。<br>
　上がりは緩やか、落ちる時は急激になりやすいため。<br>
・ほんのりと負の相関見える？　かもしれないが、単体ではとてもあてにならない<br>

In [ ]:
def psy(series, n=14):
    return (series - series.shift(1)).rolling(n).apply(lambda s:(s>=0).mean())

fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

d["サイコロジカルライン"] = psy(d.Close_adj)
# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線", color="orange")


ax2.plot(d.Date, d["サイコロジカルライン"] , label="サイコロジカルライン", color="red")

# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('サイコロジカルライン（14日）')

# gridの設定
plt.grid()
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)
plt.show()

print("サイコロジカルライン（14日）　統計量")
display(d["サイコロジカルライン"].describe())
d["サイコロジカルライン"].plot.box()
plt.show()

sns.boxplot(x="サイコロジカルライン", y="Target", data=d)
plt.xticks(rotation=90)
plt.show()


### シミュレーション
逆張り戦略<br>
シンプルに75％を超えたら売り、25％を割ったら買い。<br>
n=20。営業日換算で約1ヶ月分。やや長めの値を設定<br>
n=14などだと、サイコロジカルラインの取る値も1/nのため、細かなパラメタが意味を持たない。<br>
例:しきい値65％と70％が同じになってしまう（9/14 < 65% < 70% < 10/14のため）

In [ ]:
%%time
def simulate(d, threshold=0.25, n=14):
    series = d.Close_adj
    target = d.Target
    
    a = psy(series, n=n)

    s = pd.Series(np.where(a < 0.5 - threshold, 1,
                             np.where(a > 0.5 + threshold, -1, 0)
                         ), index=series.index, name="Cross")

    data =  pd.concat([s, target], axis=1).groupby("Cross")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
# simulate(d)
result = {}
for i in range(5):
    print(i)
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, threshold=0.15 + 0.05*i,n=20))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
n=20の場合、しきい値65%～75%で、有意水準を 0.01/5 としても有意。<br>
それ以上では数が少なすぎて、有意差となっていないと思われる。nを減らすかしきい値を緩めるかが必要。<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

### 結論
逆張り指標としてまずまず使えそうだが、nにかなり左右される。<br>
しきい値自体よりもnのほうが重要なパラメタかもしれない。<br>
しきい値は25%,75％固定で、何日前までみるかをチューニングすべきか？<br>

## 一目均衡表
1. 転換線:過去9日間の高値／安値の中間値
2. 基準線：過去26日間の高値／安値の中間値
3. 先行スパン1：転換線と基準線の中間値を26日先にずらす
4. 先行スパン2：52日間の高値と安値の中間値を26日先にずらす
5. 遅行スパン：当日終値を26日前にずらす

### 使い方
#### ①基準線と転換線の手法
・基準線が上向きなら上昇、下向きなら下降トレンド<br>
・ローソク足が基準線の上なら強い相場、下側なら弱い相場<br>
・基準線が上向きかつ転換線が基準線を上に突き抜ける（ゴールデンクロス）、あるいは基準線が下向きで、転換線が基準線を下に突き抜ける（デッドクロス）<br>
#### ②先行スパン1,2の手法
雲がレジスタンスラインあるいはサポートラインとなる<br>
・ローソク足が雲の上＝強い相場<br>
・ローソク足よりも雲が上にある＝上値抵抗線<br>
・ローソク足が雲を突き抜け＝ブレイク（上昇あるいは下降サイン）<br>
・先行スパンの交差（雲のねじれ）＝相場の転換もしくは加速<br>

#### ③遅行スパン
・遅行スパンがローソク足を上回った場合（ゴールデンクロス）＝買シグナル<br>

#### ④三役好転（逆転）
下記の３条件が揃った場合買シグナル（逆は売りシグナル）<br>
※上記の【好転】が揃った場合<br>
・転換線＞基準線<br>
・ローソク足＞雲<br>
・遅行スパン＞ローソク足<br>

本来は先行スパン1,先行スパン2の間の雲を見るのだが、データの期間が狭くてあんまり参考にならない<br>
一応終盤を見ると、雲を抜けられず、抵抗帯に弾き返されていることがわかる。

In [ ]:
fig,ax1=plt.subplots(figsize=(16,4))

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
# ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線", color="orange")

d["転換線"] = d.Close_adj.rolling(9).apply(lambda s:(s.max()+s.min())/2)
d["基準線"] = d.Close_adj.rolling(26).apply(lambda s:(s.max()+s.min())/2)
d["先行スパン1"] = d[["転換線","基準線"]].mean(axis=1).shift(25)
d["先行スパン2"] = d.Close_adj.rolling(52).apply(lambda s:(s.max()+s.min())/2).shift(25)
d["遅行スパン"] = d.Close_adj.shift(-25)

for c in ["転換線","基準線","先行スパン1","先行スパン2","遅行スパン"]:
    ax1.plot(d.Date, d[c] , label=c)

ax1.fill_between(
    d.Date,
    d["先行スパン1"],
    d["先行スパン2"],
    where=(d["先行スパン1"] >= d["先行スパン2"]),
    alpha=0.3,
    interpolate=True
)
ax1.fill_between(
    d.Date,
    d["先行スパン2"],
    d["先行スパン1"],
    where=(d["先行スパン2"] >= d["先行スパン1"]),
    alpha=0.3,
    interpolate=True
)

# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')

# gridの設定
plt.grid()
# handler1, label1 = ax1.get_legend_handles_labels()
# handler2, label2 = ax2.get_legend_handles_labels()
# ax1.legend(handler1 + handler2, label1 + label2)
plt.legend()
plt.show()


### シミュレーション
３役好転、あるいは逆転で売り買い<br>
と思ったけれど、３役好転する銘柄が少なすぎるのでシグナルの差をしきい値にして売買。<br>
※ 1⃣3⃣はほぼ排他の関係だが、2⃣のみ売り買いどちらでもない（雲の中にいる）事がありうる。<br>
#### ①基準線と転換線の手法
・基準線が上向きなら上昇、下向きなら下降トレンド<br>
・ローソク足が基準線の上なら強い相場、下側なら弱い相場<br>
・基準線が上向きかつ転換線が基準線を上に突き抜ける（ゴールデンクロス）、あるいは基準線が下向きで、転換線が基準線を下に突き抜ける（デッドクロス）<br>
#### ②先行スパン1,2の手法
雲がレジスタンスラインあるいはサポートラインとなる<br>
・ローソク足が雲の上＝強い相場<br>
・ローソク足よりも雲が上にある＝上値抵抗線<br>
・ローソク足が雲を突き抜け＝ブレイク（上昇あるいは下降サイン）<br>
・先行スパンの交差（雲のねじれ）＝相場の転換もしくは加速<br>

#### ③遅行スパン
・遅行スパンがローソク足を上回った場合（ゴールデンクロス）＝買シグナル<br>

#### ④三役好転（逆転）
下記の３条件が揃った場合買シグナル（逆は売りシグナル）<br>
※上記の【好転】が揃った場合<br>
・転換線＞基準線<br>
・ローソク足＞雲<br>
・遅行スパン＞ローソク足<br>


In [ ]:
%%time
def simulate(d, threshold=2):
    series = d.Close_adj
    target = d.Target

    conv = series.rolling(9).apply(lambda s:(s.max()+s.min())/2)
    base = series.rolling(26).apply(lambda s:(s.max()+s.min())/2)
    pre1 = ((conv + base)/2).shift(25)
    pre2 = d.Close_adj.rolling(52).apply(lambda s:(s.max()+s.min())/2).shift(25)
    lagg = d.Close_adj.shift(25)
    
    buy1 = base > conv
    sell1 = base < conv
    buy2 = (series > pre1) & (series > pre2)
    sell2 = (series < pre1) & (series < pre2)
    buy3 = series > lagg
    sell3 = series < lagg
    buy = pd.concat([buy1, buy2 ,buy3], axis=1).fillna(False).sum(axis=1)
    sell = pd.concat([sell1, sell2 ,sell3], axis=1).fillna(False).sum(axis=1)
    

    s = pd.Series(np.where(buy - sell >= threshold, 1,
                             np.where(sell - buy >= threshold, -1, 0)
                         ), index=series.index, name="BuySell")

    data =  pd.concat([s, target], axis=1).groupby("BuySell")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(3):
    print(i)
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, threshold=i+1 ))

In [ ]:
import scipy.stats as stats

for i in range(3):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
しきい値1は流石に微妙。シグナルの差が2を超えたとき位を基準にするのが良さそう<br>
しきい値を3、つまり三役好転のみとすると売り買い自体ができなくて銘柄あたりの期待値が下がっている。有意水準を 0.01/5 としても有意。<br>

In [ ]:
for i in range(3):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

## ROC（Rate of Change）
過去の株価(n日前）に対する割合(百分率で表したり、1を引いて0を基準としたりする場合もあるが本質的にはどれも同じ)。<br>
シンプルに、トレンド傾向を見ることが出来る。100以上なら強気（上昇）トレンド、100以下なら弱気（下降）トレンド。100に近いほどトレンドが弱い。<br>
時系列的に見て、100以上の値から100に近づくと上昇トレンドの終焉、逆ならば下降トレンドの終焉を示す。<br>
移動平均線や一目均衡表に近い考えのため、移動平均や一目均衡表のゴールデン・デッドクロス、雲抜けと合わせて用いるとよい。<br>

In [ ]:
def roc(series, window=14):
    return series/series.shift(window) - 1

fig,ax1=plt.subplots(figsize=(16,4))
ax2=plt.twinx(ax1)

# 描画する
ax1.plot(d.Date, d.Close_adj, label="株価", color="blue")
ax1.plot(d.Date, d.Close_adj.rolling(25).mean(), label="25日移動平均線", color="orange")

d["ROC"] = roc(d.Close_adj,window=14)
ax2.plot(d.Date, d["ROC"], label="ROC(14日)", color="red")

# グラフ周りの設定
plt.title('株価推移')
ax1.set_xlabel('年月')
ax1.set_ylabel('株価')
ax2.set_ylabel('ROC(14日)')


# gridの設定
plt.grid()
handler1, label1 = ax1.get_legend_handles_labels()
handler2, label2 = ax2.get_legend_handles_labels()
ax1.legend(handler1 + handler2, label1 + label2)
plt.show()

sns.scatterplot(x="ROC", y="Target", data=d)
plt.show()


## シミュレーション
ROCの0を基準としたゴールデンクロス、デッドクロスで売買。<br>
買シグナル<br>
ROCが負の値から正の値に転換≒ゴールデンクロス<br>
※売りシグナルはその逆<br>


In [ ]:
%%time
def simulate(d, window=14):
    series = d.Close_adj
    target = d.Target
    a = roc(series, window)
    buy = (a > 0) & (a.shift(1)<0)
    sell = (a < 0) & (a.shift(1)>0)

    s = pd.Series(np.where(buy, 1,
                             np.where(sell, -1, 0)
                         ), index=series.index, name="BuySell")
    data =  pd.concat([s, target], axis=1).groupby("BuySell")[target.name].sum()
    return data[data.index>0].sum() - data[data.index<0].sum()
#simulate(d)
result = {}
for i in range(5):
    print(i)
    result[i] = df.groupby("SecuritiesCode").apply(lambda _d: simulate(_d, window=10 + i*2 ))

In [ ]:
import scipy.stats as stats

for i in range(5):
    print(i,(result[i]!=0).sum())
    print("合計",result[i].sum())
    print("平均",result[i][result[i]!=0].mean())
    result[i][result[i]!=0].hist(bins=np.arange(50)/100 - 0.25)
    plt.show()
    
    fig = plt.subplots(figsize=(8,8)) 
    stats.probplot(result[i][result[i]!=0], dist="norm", plot=plt)
    plt.show()
    print("S-W検定",stats.shapiro(result[i]))
    print("K-S検定",stats.kstest(result[i], "norm"))


### 検証
統計的に有意な差かどうか検証。上記から正規分布とは仮定できそうにない。<br>
ノンパラメトリックに勝ち負けで考えてみる。<br>
2000銘柄で、正（勝ち）、負（負け）が偶然起こる範囲かどうか。<br>
ウィルコクソンの符号付き順位和検定<br>
12日前が一番良いという結果。他（10,14,16,18）ではいまいち結果が出ていない(有意水準0.01/5とする)<br>

In [ ]:
for i in range(5):
    print(i, result[i][result[i]!=0].mean())
    print(stats.wilcoxon(result[i], y=None, zero_method='wilcox', correction=False))

### 結論
12日前を基準にして、ゴールデンクロス、デッドクロスを判定するのがよい。
暫定的にn=12として今後は用いることにする。<br>
10日などなら同じ曜日になりやすいからなどの仮説も立つが、この結果は次の仮説に困る。<br>